In [55]:
import os
import requests
import json
import re
import pandas as pd
import time
import matplotlib.pyplot as plt
from os.path import join
from slugify import slugify
from bs4 import BeautifulSoup
from bs4.element import NavigableString

base_dir = "nescafe-rating"
html_dir = "nescafe-rating/html"
if not os.path.exists(base_dir):
    os.makedirs(base_dir)
if not os.path.exists(html_dir):
    os.makedirs(html_dir)

In [2]:
base_url = "https://www.nescafe.co.uk/products/gold/nescafé-gold-blend-instant-coffee/?bvstate=pg:%d/ct:r"

In [56]:
# This didn't work: 

reviewers = []
ratings = []
dates = []
titles = []
descriptions = []

for i in range(0,20):
    url = base_url % (i+1)
    r = requests.get(url)
    print(url)
    soup = BeautifulSoup(r.text, "lxml")  
    reviewsSection = soup.find('div',  {'id': 'bvseo-reviewsSection'})
    
    current_reviewers = []
    current_ratings = []
    current_dates = []
    current_titles = []
    current_descriptions = []
    
    soup_f = "soup-%d.html" % (i+1)
    with open(join(html_dir, soup_f), "w") as f:
        f.write(soup.prettify())
    
    if reviewsSection:
        reviews = reviewsSection.findAll('div', {'class':'bvseo-review'})
        for review in reviews:
            author = review.find('span', {"itemprop":"author"}).text.strip()
            datePublished = review.find('meta', {"itemprop":"datePublished"}).get('content').strip()
            ratingValue = int(review.find('span', {"itemprop":"ratingValue"}).text.strip())
            title = review.find('span', {"itemprop":"name"}).text.strip()
            description = review.find('span', {"itemprop":"description"}).text.strip()
            
            current_reviewers.append(author)
            current_ratings.append(ratingValue)
            current_dates.append(datePublished)
            current_titles.append(title)
            current_descriptions.append(description)
            
        if len(current_reviewers) > 0 and len(reviewers) > 0 and current_reviewers[-1] == reviewers[-1]: 
            print("break")
            break
        
        reviewers.extend(current_reviewers)
        ratings.extend(current_ratings)
        dates.extend(current_dates)
        titles.extend(current_titles)
        descriptions.extend(current_descriptions)
        time.sleep(3)
        print("%4d: (%4d)" % (i+1, len(reviewers)), end='|')
    
    else:
        print("No reviews", soup_f)
        
print("\nFound", len(reviewers))

nescafe_reviews = pd.DataFrame({
    'reviewer' : reviewers,
    'rating': ratings,
    'date': dates
})
nescafe_reviews.head()

https://www.nescafe.co.uk/products/gold/nescafé-gold-blend-instant-coffee/?bvstate=pg:1/ct:r
   1: (   8)|https://www.nescafe.co.uk/products/gold/nescafé-gold-blend-instant-coffee/?bvstate=pg:2/ct:r
   2: (  38)|https://www.nescafe.co.uk/products/gold/nescafé-gold-blend-instant-coffee/?bvstate=pg:3/ct:r
   3: (  68)|https://www.nescafe.co.uk/products/gold/nescafé-gold-blend-instant-coffee/?bvstate=pg:4/ct:r
   4: (  98)|https://www.nescafe.co.uk/products/gold/nescafé-gold-blend-instant-coffee/?bvstate=pg:5/ct:r
   5: ( 128)|https://www.nescafe.co.uk/products/gold/nescafé-gold-blend-instant-coffee/?bvstate=pg:6/ct:r
   6: ( 158)|https://www.nescafe.co.uk/products/gold/nescafé-gold-blend-instant-coffee/?bvstate=pg:7/ct:r
   7: ( 188)|https://www.nescafe.co.uk/products/gold/nescafé-gold-blend-instant-coffee/?bvstate=pg:8/ct:r
   8: ( 218)|https://www.nescafe.co.uk/products/gold/nescafé-gold-blend-instant-coffee/?bvstate=pg:9/ct:r
   9: ( 248)|https://www.nescafe.co.uk/products/gold/nescaf

,date,rating,reviewer
0,2017-11-03,3,Bruciethefish
1,2017-11-18,1,Lottiesmum
2,2017-10-27,1,Polar Bear
3,2017-10-30,1,Neil W R
4,2017-10-19,1,Bertie


In [51]:
#Examine query strings found while examining the network analysis on the nescafe page

from urllib.parse import urlparse, parse_qs

first = urlparse("https://api.bazaarvoice.com/data/batch.json?passkey=ayhd1leg5c5ng6qr66roucnje&apiversion=5.5&displaycode=19253-en_gb&resource.q0=products&filter.q0=id%3Aeq%3A19699905-6be2-4632-a181-d1ea98006500&stats.q0=reviews&filteredstats.q0=reviews&filter_reviews.q0=contentlocale%3Aeq%3Aen_GB&filter_reviewcomments.q0=contentlocale%3Aeq%3Aen_GB&resource.q1=reviews&filter.q1=isratingsonly%3Aeq%3Afalse&filter.q1=productid%3Aeq%3A19699905-6be2-4632-a181-d1ea98006500&filter.q1=contentlocale%3Aeq%3Aen_GB&sort.q1=relevancy%3Aa1&stats.q1=reviews&filteredstats.q1=reviews&include.q1=authors%2Cproducts%2Ccomments&filter_reviews.q1=contentlocale%3Aeq%3Aen_GB&filter_reviewcomments.q1=contentlocale%3Aeq%3Aen_GB&filter_comments.q1=contentlocale%3Aeq%3Aen_GB&limit.q1=8&offset.q1=0&limit_comments.q1=3&callback=BV._internal.dataHandler0")
                 "https://api.bazaarvoice.com/data/batch.json?passkey=ayhd1leg5c5ng6qr66roucnje&apiversion=5.5&displaycode=19253-en_gb&resource.q0=products&filter.q0=id%3Aeq%3A251208d3-dc54-436d-90a7-15e4feb7bef4&stats.q0=reviews&filteredstats.q0=reviews&filter_reviews.q0=contentlocale%3Aeq%3Aen_GB&filter_reviewcomments.q0=contentlocale%3Aeq%3Aen_GB&resource.q1=reviews&filter.q1=isratingsonly%3Aeq%3Afalse&filter.q1=productid%3Aeq%3A251208d3-dc54-436d-90a7-15e4feb7bef4&filter.q1=contentlocale%3Aeq%3Aen_GB&sort.q1=relevancy%3Aa1&stats.q1=reviews&filteredstats.q1=reviews&include.q1=authors%2Cproducts%2Ccomments&filter_reviews.q1=contentlocale%3Aeq%3Aen_GB&filter_reviewcomments.q1=contentlocale%3Aeq%3Aen_GB&filter_comments.q1=contentlocale%3Aeq%3Aen_GB&limit.q1=8&offset.q1=0&limit_comments.q1=3&callback=BV._internal.dataHandler0"
second = urlparse("https://api.bazaarvoice.com/data/batch.json?passkey=ayhd1leg5c5ng6qr66roucnje&apiversion=5.5&displaycode=19253-en_gb&resource.q0=reviews&filter.q0=isratingsonly%3Aeq%3Afalse&filter.q0=productid%3Aeq%3A19699905-6be2-4632-a181-d1ea98006500&filter.q0=contentlocale%3Aeq%3Aen_GB&sort.q0=relevancy%3Aa1&stats.q0=reviews&filteredstats.q0=reviews&include.q0=authors%2Cproducts%2Ccomments&filter_reviews.q0=contentlocale%3Aeq%3Aen_GB&filter_reviewcomments.q0=contentlocale%3Aeq%3Aen_GB&filter_comments.q0=contentlocale%3Aeq%3Aen_GB&limit.q0=30&offset.q0=8&limit_comments.q0=3&resource.q1=products&filter.q1=id%3Aeq%3A19699905-6be2-4632-a181-d1ea98006500&stats.q1=reviews&filteredstats.q1=reviews&filter_reviews.q1=contentlocale%3Aeq%3Aen_GB&filter_reviewcomments.q1=contentlocale%3Aeq%3Aen_GB&callback=bv_1111_44680")
third = urlparse("https://api.bazaarvoice.com/data/batch.json?passkey=ayhd1leg5c5ng6qr66roucnje&apiversion=5.5&displaycode=19253-en_gb&resource.q0=reviews&filter.q0=isratingsonly%3Aeq%3Afalse&filter.q0=productid%3Aeq%3A19699905-6be2-4632-a181-d1ea98006500&filter.q0=contentlocale%3Aeq%3Aen_GB&sort.q0=relevancy%3Aa1&stats.q0=reviews&filteredstats.q0=reviews&include.q0=authors%2Cproducts%2Ccomments&filter_reviews.q0=contentlocale%3Aeq%3Aen_GB&filter_reviewcomments.q0=contentlocale%3Aeq%3Aen_GB&filter_comments.q0=contentlocale%3Aeq%3Aen_GB&limit.q0=30&offset.q0=38&limit_comments.q0=3&resource.q1=products&filter.q1=id%3Aeq%3A19699905-6be2-4632-a181-d1ea98006500&stats.q1=reviews&filteredstats.q1=reviews&filter_reviews.q1=contentlocale%3Aeq%3Aen_GB&filter_reviewcomments.q1=contentlocale%3Aeq%3Aen_GB&callback=bv_1111_18875")
                 

q_first = parse_qs(first.query)
q_second = parse_qs(second.query)
q_third = parse_qs(third.query)

first_attrs = sorted(list(q_first.keys()))
second_attrs = sorted(list(q_second.keys()))
third_attrs = sorted(list(q_third.keys()))

for one,two,three in zip(first_attrs, second_attrs, third_attrs):
    if one == two and two == three:
        print(one, q_first[one], q_second[two], q_third[three])

apiversion ['5.5'] ['5.5'] ['5.5']
callback ['BV._internal.dataHandler0'] ['bv_1111_44680'] ['bv_1111_18875']
displaycode ['19253-en_gb'] ['19253-en_gb'] ['19253-en_gb']
filter.q0 ['id:eq:19699905-6be2-4632-a181-d1ea98006500'] ['isratingsonly:eq:false', 'productid:eq:19699905-6be2-4632-a181-d1ea98006500', 'contentlocale:eq:en_GB'] ['isratingsonly:eq:false', 'productid:eq:19699905-6be2-4632-a181-d1ea98006500', 'contentlocale:eq:en_GB']
filter.q1 ['isratingsonly:eq:false', 'productid:eq:19699905-6be2-4632-a181-d1ea98006500', 'contentlocale:eq:en_GB'] ['id:eq:19699905-6be2-4632-a181-d1ea98006500'] ['id:eq:19699905-6be2-4632-a181-d1ea98006500']
filter_reviewcomments.q0 ['contentlocale:eq:en_GB'] ['contentlocale:eq:en_GB'] ['contentlocale:eq:en_GB']
filter_reviewcomments.q1 ['contentlocale:eq:en_GB'] ['contentlocale:eq:en_GB'] ['contentlocale:eq:en_GB']
filter_reviews.q0 ['contentlocale:eq:en_GB'] ['contentlocale:eq:en_GB'] ['contentlocale:eq:en_GB']
filter_reviews.q1 ['contentlocale:eq:en_

In [45]:
main_dictionary = q_first
main_dictionary["limit.q1"] = 30
main_dictionary["offset.q1"] = 0

total = 0
r = requests.get('https://api.bazaarvoice.com/data/batch.json',  params = main_dictionary)

content = r.text
first_parentheses = content.index("(") +1
content = content[first_parentheses:-1]
j = json.loads(content)
total_results = int(j["BatchedResults"]["q1"]["TotalResults"])

Results 1926


In [61]:

reviews = []

products = {'NESCAFÉ Fine Blend':    '251208d3-dc54-436d-90a7-15e4feb7bef4',
    'NESCAFÉ Original'  :    '5345a1a9-0d19-4fca-aaae-bf7f6c72efdd',
    'NESCAFÉ Original 3in1': '6bd8c316-5d2a-4c3a-b766-4f74d13e86d0',
    'NESCAFÉ Original 3in1 Strong': 'f00510b2-c27d-46d0-9018-35e950e7797a',
    'NESCAFÉ Original Refill':'5345a1a9-0d19-4fca-aaae-bf7f6c72efdd',
    'NESCAFÉ Original 3in1 Extra Smooth': 'e2506e71-494b-4114-b288-067a97314bff',
    'NESCAFÉ Original 2in1':'86ca6997-2f84-454e-8121-5dd80a3da883',
    'NESCAFÉ Original Decaff': '40640e88-8830-4a94-b793-8445e685a1d5',
    'NESCAFÉ GOLD BLEND BARISTA STYLE':'2bc30ce6-ed77-4f95-bff5-5483fbc9c171',
    'NESCAFÉ GOLD BLEND Decaff Refill':'b528ed08-e3cf-4dcf-bbca-7f219fac2ae3',
    'NESCAFÉ GOLD BLEND Decaff': 'a47fd703-64b6-4eae-8ffc-d1938dee5099',
    'NESCAFÉ GOLD Black' : 'cb4d59cf-c48d-47ef-a943-50b2ae5d01ee',
    'NESCAFÉ BLEND 37':'2adb8f53-2da9-4a0d-8340-d509f3c6883d',
    'NESCAFÉ GOLD Crema': '0f4a6269-1dd9-4337-8921-04897b5d34c4',
    'NESCAFÉ GOLD BLEND':    '19699905-6be2-4632-a181-d1ea98006500',
    'NESCAFÉ GOLD BLEND Refill':'ce06204b-3a6c-44a9-8a46-837c355ef4c7',
    'NESCAFÉ Alta Rica':  'c4006dc8-8368-4210-a462-146e71ac44b8',
    'NESCAFÉ Alta Rica1': '6eaa0fb9-6aaa-4eeb-8c54-548f8847ad49',
    'NESCAFÉ Cap Colombie':'bafcae76-5ec8-40e2-a2c1-f38cbb200280',
    'NESCAFÉ Espresso': '80c2ac80-5551-479c-a069-c21309b00304',
    'NESCAFÉ AZERA Americano': 'd68fda79-d489-4d8e-a19f-f26def87180d',
    'NESCAFÉ AZERA Intenso': '59f60618-0335-4b74-9e52-1c58c980e454',
    'NESCAFÉ AZERA Espresso': '47150cc9-2556-4a61-be9a-6dd4bb1e7eef',
    'NESCAFÉ AZERA Cappuccino': 'bf9b9f4b-0b88-4e07-8d02-95d6c745824f',
    'NESCAFÉ AZERA Latte': 'b918ef88-6ccf-49ee-aaaf-915ac4711c75',
    'NESCAFÉ AZERA Americano Decaff': '8294c028-0adc-4d65-b3af-4126d1eb4883',
    'NESCAFÉ AZERA Coffee To Go Americano': '273dcfb3-fce1-413c-91d2-1ad34ceeb8da',
    'NESCAFÉ AZERA Coffee To Go Latte': '492d4fe0-7a4e-4c14-8ca6-54e602cf32c4'
}

for p in products:
    print("Getting reviews for", p)
    offset = 0
    limit = 30
    
    productid = products[p]
    main_dictionary["filter.q1"] = ['isratingsonly:eq:false', 
                                    'productid:eq:' + productid, 
                                    'contentlocale:eq:en_GB'] 

    while True:
        main_dictionary["limit.q1"] = limit
        main_dictionary["offset.q1"] = offset

        r = requests.get('https://api.bazaarvoice.com/data/batch.json',  params = main_dictionary)
        content = r.text
        first_parentheses = content.index("(") +1
        content = content[first_parentheses:-1]
        j = json.loads(content)

        folder = join(base_dir, slugify(p))
        if not os.path.exists(folder):
            os.makedirs(folder)
        with open(join(folder, "reviews-limit-%d.json" % offset), "w") as file:
            file.write(json.dumps(j, indent=4))

        results = j["BatchedResults"]["q1"]["Results"]
        reviews.extend(results)

        offset = offset + 30
        if len(results) != limit:
            break
print("Done getting reviews!")

Getting reviews for NESCAFÉ Fine Blend
Getting reviews for NESCAFÉ Original
Getting reviews for NESCAFÉ Original 3in1
Getting reviews for NESCAFÉ Original 3in1 Strong
Getting reviews for NESCAFÉ Original Refill
Getting reviews for NESCAFÉ Original 3in1 Extra Smooth
Getting reviews for NESCAFÉ Original 2in1
Getting reviews for NESCAFÉ Original Decaff
Getting reviews for NESCAFÉ GOLD BLEND BARISTA STYLE
Getting reviews for NESCAFÉ GOLD BLEND Decaff Refill
Getting reviews for NESCAFÉ GOLD BLEND Decaff
Getting reviews for NESCAFÉ GOLD Black
Getting reviews for NESCAFÉ BLEND 37
Getting reviews for NESCAFÉ GOLD Crema
Getting reviews for NESCAFÉ GOLD BLEND
Getting reviews for NESCAFÉ GOLD BLEND Refill
Getting reviews for NESCAFÉ Alta Rica
Getting reviews for NESCAFÉ Alta Rica1
Getting reviews for NESCAFÉ Cap Colombie
Getting reviews for NESCAFÉ Espresso
Getting reviews for NESCAFÉ AZERA Americano
Getting reviews for NESCAFÉ AZERA Intenso
Getting reviews for NESCAFÉ AZERA Espresso
Getting revi